# Summary
This document corresponds to Exercise 2 of [this file](https://github.com/PerformanceEstimation/Learning-Performance-Estimation/blob/main/Course.pdf).

The goal of this exercise is to compute a closed-form for the worst-case ratio $\frac{\|x_{k+1}-x_\star\|^2}{\|x_k-x_\star\|^2}$ when $x_\star=\textrm{argmin}_x f(x)$ for some $f$ that is $L$-smooth and $\mu$-strongly convex, and where $x_{k+1}=x_k-\gamma_k \nabla f(x_k)$ is obtained from a gradient step (with stepsize $\gamma_k$) from $x_k$.


We first install a few packages easing convex optimization and symbolic computations in Python.

In [ ]:
!pip install sympy
!pip install cvxpy

### Exercises 2.1 & 2.2

In exercises 2.1 and 2.2, there is no numerical experiments.

### Exercise 2.3: trick for obtaining the LMI using symbolic computation

For doing this, let us first define a few symbols using SymPyimport sympy as sm.


In [ ]:
import sympy as sm

# create symbols for the problem parameters:
L = sm.Symbol('L')
mu = sm.Symbol('mu')
gamma = sm.Symbol('gamma')

# create symbols for the "primal" variables:
x0 = sm.Symbol('x0')
g0 = sm.Symbol('g0')
f0 = sm.Symbol('f0')
xs = 0 # wlog, x_* = 0
gs = 0 # constraint g_* = 0
fs = 0 # wlog, f_* = 0
x1 = x0 - gamma * g0 # define x1 using previous symbols:

# create symbols for the "dual" variables
rho = sm.Symbol('rho')
l1 = sm.Symbol('l1')
l2 = sm.Symbol('l2')

Write all inequalities (in the form "$...\leqslant 0$") and the Lagrangian of the problem:

In [ ]:
# the two interpolation constraints in the form "constraint <= 0"
constraint1 = f0 - fs + g0*(xs-x0) + 1/2/L * g0**2 + mu/(2*(1-mu/L)) * (x0-xs-1/L*g0)**2
constraint2 = fs - f0 + 1/2/L * g0**2 + mu/(2*(1-mu/L)) * (x0-xs-1/L*g0)**2
# the objective and the "initial condition" constraint: (also in the form "constraint <= 0")
primal_objective = (x1-xs)**2
initial_condition = (x0-xs)**2-1

# Lagrangian:
Lagrangian = - l1*constraint1 - l2*constraint2 - rho * initial_condition + primal_objective

Now, perform the simple trick: obtain the LMI and the linear constraint (i.e., the dual constraints) from differentiating the Lagrangian

In [ ]:
# This is the LMI:
LMI = sm.simplify(sm.hessian( -Lagrangian , (x0,g0))/2) 
# This is the linear constraint ==0 in the LMI
LinearConst = sm.simplify(sm.diff(-Lagrangian,f0))

Substitute $\lambda_2$ for getting to the same dual formulation:

In [ ]:
# For getting the same LMI as in the document, substitute l2 by l1 and simplify
LMI_simplified = sm.simplify(LMI.subs(l2,l1))

In [ ]:
LMI_simplified # show the LMI (>=0)

In [ ]:
LinearConst # show linear constraint (==0)

### Exercise 2.4: numerical solutions to the LMI

For getting a few hints on how to solve this LMI in closed-form, a good practice is to first try it numerically for a few values of $\mu$, $L$, and $\gamma_k$.

The following code implements the LMI.


In [ ]:
import cvxpy as cp
def lmi_convergence_distance(L, mu, gamma):

    # Write the LMI.
    S = cp.Variable((2, 2)) # this is the matrix S
    l1 = cp.Variable()      # this is the multiplier (lambda1 == lambda2) which we denote l1
    tau = cp.Variable()     # this is the objective
    
    s11 = tau-1+l1*L*mu/(L-mu)
    s12 = gamma-l1*(L+mu)/2/(L-mu)
    s22 = l1/(L-mu)-gamma**2
    
    constraints = [S >> 0,
                   S[0,0] == s11,
                   S[1,1] == s22,
                   S[0,1] == s12,
                   S[1,0] == s12,
                   l1 >= 0]
    
    prob = cp.Problem(cp.Minimize(tau), constraints)
    prob.solve()
    return tau.value, l1.value, S.value

The following code solves the LMI for a grid of $\gamma_k$'s.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nb_test = 50
mu = .1
L = 1
gamma = np.linspace(-1., 3., num=nb_test)

l1_list = list()
tau_list = list()
S_list = list()

for i in range(nb_test):
    tau,l1,S = lmi_convergence_distance(L, mu, gamma[i])
    l1_list.append(l1[()])
    tau_list.append(tau[()])
    S_list.append(S)

Plot the output!

In [ ]:
plt.plot(gamma, tau_list, '.-',label='$\tau_1$')
plt.plot(gamma, l1_list, '.-',label='$\lambda_1$')

plt.xlabel('$\gamma$')
plt.ylabel('Multipliers')
plt.legend()
plt.show()

Recover useful information about $S$: its eigenvalues.

In [ ]:
firsteig_list = list()
seceig_list = list()

for i in range(nb_test):
    eigsV, _ = np.linalg.eigh(S_list[i])
    firsteig_list.append(np.max(eigsV))
    seceig_list.append(np.min(eigsV))

plt.plot(gamma, firsteig_list, '.-',label='first eigenvalue')
plt.plot(gamma, seceig_list, '.-',label='second eigenvalue')

plt.xlabel('$\gamma$')
plt.ylabel('$\Lambda(S)$')
plt.legend()
plt.show()

In case you have a candidate expression for $\tau(\mu,L,\gamma_k)$, compare it with the numerics?
(you can come back to this cell a bit later with a few expressions to test)

In [ ]:
tau_candidates = [ 1 for i in range(nb_test)] # replace "1" by an appropriate candidate.
plt.plot(gamma, tau_list, '.-',label='LMI')
plt.plot(gamma, tau_candidates, '--',label='TRIAL')

plt.xlabel('$\gamma$')
plt.ylabel('Multipliers')
plt.legend()

plt.show()

### Exercise 2.5: getting closed-form solutions to the LMI

As we saw numerically, the matrix $S$ has rank $1$ for most values of the step-sizes!
As the problem of finding a bound on the convergence rate $\frac{\|x_{k+1}-x_\star\|^2}{\|x_k-x_\star\|^2}$ corresponds to a linear problem with an LMI constraint, it is natural that the optimal solution is on the boundary of the feasible set, and we can use that for solving the problem in closed-form.

We use a bit of symbolic computation below for simplicity.

The following cell encodes a symbolic matrix $S$:

In [ ]:
import sympy as sm

tau = sm.Symbol('tau')
l1 = sm.Symbol('l1')
gamma = sm.Symbol('gamma')
L = sm.Symbol('L')
mu = sm.Symbol('mu')

S = sm.Matrix([[tau-1+l1*L*mu/(L-mu), gamma-l1*(L+mu)/2/(L-mu)], [gamma-l1*(L+mu)/2/(L-mu), l1/(L-mu)-gamma**2]])

For making $S$ rank defficient, let's choose $\tau$ for cancelling out the determinant:

In [ ]:
candidate_tau=sm.solve(S.det(),tau)
candidate_tau[0]

There are two possibilities for choosing $\lambda_1$:
- on the boundary of the PSD cone ($S=0$),
- minimize $\tau$ (and verify that the corresponding $(\lambda_1,\tau,S)$ is feasible for the LMI afterwards).

Because we observed numerically that $S$ was rank $1$ for most choices of stepsizes, let's focus on the second possibility.

In [ ]:
dtau=sm.simplify(sm.diff(candidate_tau[0],l1)) #differentiate tau with respect to lambda_1
dtau

In [ ]:
l1sol=sm.solve(dtau,l1) # solve dtau/dlambda_1 == 0 in lambda1
l1sol # show the two possibilities!

The corresponding "final" expressions for $\tau$ that must be checked are therefore:

In [ ]:
exprtau1=sm.simplify(candidate_tau[0].subs(l1,l1sol[0]))
exprtau1.factor()

In [ ]:
exprtau2=sm.simplify(candidate_tau[0].subs(l1,l1sol[1]))
exprtau2.factor()

### Exercise 2.6: writing a proof without semidefinite programming / LMI

This exercise requires no coding.

### Exercise 2.7: changing the ratio to $\frac{\|\nabla f(x_{k+1})\|^2}{\|\nabla f(x_k)\|^2}$

Using similar steps as before, solve the LMI for finding the worst-case value of the ratio $\frac{\|\nabla f(x_{k+1})\|^2}{\|\nabla f(x_k)\|^2}$: (1) numerically, and (2) in closed-form.

### Exercise 2.8: changing the ratio to $\frac{f(x_{k+1})-f_\star}{f(x_k)-f_\star}$

Using similar steps as before, solve the LMI for finding the worst-case value of the ratio $\frac{f(x_{k+1})-f_\star}{f(x_k)-f_\star}$: (1) numerically, and (2) in closed-form.

For convenience, a code is provided below.

In [ ]:
# Import packages.
import cvxpy as cp

def lmi_functionvalues(L, mu, gamma):

    # Write the LMI.
    S = cp.Variable((3, 3))
    lamb = cp.Variable(6)
    tau = cp.Variable()

    s11 = mu*L/(L-mu) * (lamb[0]+lamb[1]+lamb[2]+lamb[3])
    s12 = -L/(L-mu) * (lamb[1]+gamma*mu*(lamb[2]+lamb[3])) - mu/(L-mu)*lamb[0]
    s13 = -1/(L-mu) * (L*lamb[3]+mu*lamb[2])
    s22 = 1/(L-mu) * (gamma*mu*(gamma*L*(lamb[2]+lamb[3]+lamb[4]+lamb[5])-2*lamb[4])-2*gamma*L*lamb[5]+lamb[0]+lamb[1]+lamb[4]+lamb[5])
    s23 = 1/(L-mu) * (gamma*L*lamb[3]+lamb[4]*(gamma*L-1)+gamma*mu*(lamb[2]+lamb[5])-lamb[5])
    s33 = 1/(L-mu) * (lamb[2]+lamb[3]+lamb[4]+lamb[5])
    constraints = [0==tau-lamb[0]+lamb[1]-lamb[4]+lamb[5],
                   1==-lamb[2]+lamb[3]+lamb[4]-lamb[5],
                   S >> 0,
                   S[0,0] == s11,
                   S[1,1] == s22,
                   S[2,2] == s33,
                   S[0,1] == s12,
                   S[1,0] == s12,
                   S[0,2] == s13,
                   S[2,0] == s13,
                   S[1,2] == s23,
                   S[2,1] == s23,
                   lamb >= 0]
    # trick to try later: impose the additional
    # lamb[0]==0,
    # lamb[2]==0
    # lamb[5]==0
    
    prob = cp.Problem(cp.Minimize(tau), constraints)
    prob.solve()
    return tau.value, lamb.value, S.value

The following code plots the numerical values of the multipliers:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

nb_test = 50

mu = .1
L = 1
gamma = np.linspace(0, 2., num=nb_test)
verbose = 0
taus = np.empty([nb_test])
lambdas = np.empty([6, nb_test])
S_list = list()

for i in range(nb_test):
    tau, lamb,S = lmi_functionvalues(L=L, mu=mu, gamma=gamma[i])
    taus[i]=tau[()]
    lambdas[:,i]=lamb
    S_list.append(S)


plt.plot(gamma, lambdas[0,:], '-',label='$\lambda_1$')
plt.plot(gamma, lambdas[1,:], '-',label='$\lambda_2$')
plt.plot(gamma, lambdas[2,:], '-',label='$\lambda_3$')
plt.plot(gamma, lambdas[3,:], '-',label='$\lambda_4$')
plt.plot(gamma, lambdas[4,:], '-',label='$\lambda_5$')
plt.plot(gamma, lambdas[5,:], '-',label='$\lambda_6$')
plt.legend()
plt.xlabel('$\gamma$')
plt.ylabel('Multipliers')

plt.show()

In [ ]:
eigsV = np.empty([3, nb_test])

for i in range(nb_test):
    eigsV[:,i], _ = np.linalg.eigh(S_list[i])

plt.plot(gamma, eigsV[0,:], '.-')
plt.plot(gamma, eigsV[1,:], '.-')
plt.plot(gamma, eigsV[2,:], '.-')

plt.xlabel('$\gamma$')
plt.ylabel('$\Lambda(S)$')
plt.show()

### Exercise 2.9: obtain the LMI formulations from exercises 2.7 and 2.8 using symbolic computation.

By adapting the idea from exercise 2.3, obtain the two LMI formulations for the different ratios.